In [39]:
from pymongo import MongoClient
mongo_server = '52.14.175.152'

 1) 위의 예제에서 주별 1000만 이사의 인구를 가지는 주를 필터링 하는 예제를 aggregation을 쓰지않고 직접 python으로 구현해보세요.

In [40]:
mongo = MongoClient (mongo_server, 54329)
col = mongo.mydata.zip


results = col.find()

    
    
d = dict()

for doc in results:
    z = doc['state'] # state를 key로 지정
    if z not in d:
        d[z] = doc["pop"] # 인구수를 value로 
    else:
        d[z] = d[z] + doc['pop'] #기존에 state가 있으면 pop만 더함
        
for state in d:
    if d[state] >= 10*1000*1000:
        print state, d[state]
       

FL 12686644
TX 16984601
NY 17990402
PA 11881643
CA 29754890
IL 11427576
OH 10846517


 2)  import한 restaurant 데이터를 가지고 각 borough 마다 레스토랑이 몇 개있는지 계산하시오. (완)

In [41]:
mongo = MongoClient (mongo_server, 54329)
col = mongo.mydata.restaurant


pipelines = []
pipelines.append({'$group':{'_id':'$borough', 'total':{'$sum':1} }})

result = col.aggregate(pipelines)

for doc in result:
    print doc

{u'total': 51, u'_id': u'Missing'}
{u'total': 969, u'_id': u'Staten Island'}
{u'total': 6086, u'_id': u'Brooklyn'}
{u'total': 2338, u'_id': u'Bronx'}
{u'total': 5656, u'_id': u'Queens'}
{u'total': 10259, u'_id': u'Manhattan'}


 3) address의 zipcode별로 'Brazilian' 레스토랑이 몇개 있는지 계산하시오 (완)

In [42]:
mongo = MongoClient (mongo_server, 54329)
col = mongo.mydata.restaurant


pipelines = []
pipelines.append({'$match':{"cuisine":'Brazilian'}})
pipelines.append({'$group':{'_id':{'zipcode':'$address.zipcode'}, 'no of Brazilian':{'$sum':1}}})


result = col.aggregate(pipelines)

for doc in result:
    print doc

{u'no of Brazilian': 1, u'_id': {u'zipcode': u'11238'}}
{u'no of Brazilian': 3, u'_id': {u'zipcode': u'11106'}}
{u'no of Brazilian': 1, u'_id': {u'zipcode': u'10002'}}
{u'no of Brazilian': 2, u'_id': {u'zipcode': u'10014'}}
{u'no of Brazilian': 2, u'_id': {u'zipcode': u'11101'}}
{u'no of Brazilian': 1, u'_id': {u'zipcode': u'11377'}}
{u'no of Brazilian': 1, u'_id': {u'zipcode': u'10034'}}
{u'no of Brazilian': 1, u'_id': {u'zipcode': u'11368'}}
{u'no of Brazilian': 1, u'_id': {u'zipcode': u'10009'}}
{u'no of Brazilian': 4, u'_id': {u'zipcode': u'10036'}}
{u'no of Brazilian': 1, u'_id': {u'zipcode': u'10128'}}
{u'no of Brazilian': 3, u'_id': {u'zipcode': u'10019'}}
{u'no of Brazilian': 2, u'_id': {u'zipcode': u'10065'}}
{u'no of Brazilian': 2, u'_id': {u'zipcode': u'11211'}}
{u'no of Brazilian': 1, u'_id': {u'zipcode': u'11103'}}


 4) 평균평점이 가장 높은 레스토랑을 5개 추출하세요 (완)

In [43]:
from pprint import pprint

mongo = MongoClient (mongo_server, 54329)
col = mongo.mydata.restaurant


# 이건 그냥 1위 레스토랑의 세부 사항 출력---------------
finds = col.find({"name" : 'Juice It Health Bar'})
for doc in finds:
    pprint (doc)
    print 
# ------------------------------------------------------    
    
pipelines = []

pipelines.append({'$project': {'_id':'$restaurant_id','name':'$name', 
                               'borough':'$borough',
                               'average': {'$avg':"$grades.score"}}}) #각 레스토랑의 평균 구함
pipelines.append({'$sort':{'average':-1}}) # 평균을 내림차순으로 정리 함
pipelines.append({'$limit':5} )  # 5개까지


result = col.aggregate(pipelines)


for doc in result:
    print doc

{u'_id': ObjectId('596efb666490e9138ab279eb'),
 u'address': {u'building': u'1068',
              u'coord': [-73.958167, 40.681357],
              u'street': u'Fulton Street',
              u'zipcode': u'11238'},
 u'borough': u'Brooklyn',
 u'cuisine': u'Juice, Smoothies, Fruit Salads',
 u'grades': [{u'date': datetime.datetime(2015, 1, 20, 0, 0),
              u'grade': u'Not Yet Graded',
              u'score': 75}],
 u'name': u'Juice It Health Bar',
 u'restaurant_id': u'50015959'}

{u'average': 75.0, u'_id': u'50015959', u'borough': u'Brooklyn', u'name': u'Juice It Health Bar'}
{u'average': 73.0, u'_id': u'50017374', u'borough': u'Bronx', u'name': u'Golden Dragon Cuisine'}
{u'average': 69.0, u'_id': u'50018457', u'borough': u'Brooklyn', u'name': u"Chelsea'S Juice Factory"}
{u'average': 69.0, u'_id': u'50015245', u'borough': u'Bronx', u'name': u'Palombo Pastry Shop'}
{u'average': 65.0, u'_id': u'41703965', u'borough': u'Manhattan', u'name': u'Go Go Curry'}


 5) cusine별로 평점이 가장 높은 레스토랑을 추출하세요

In [74]:
from pprint import pprint
#3
mongo = MongoClient (mongo_server, 54329)
col = mongo.mydata.restaurant

result = col.find()

pipelines = []

pipelines.append({'$project': {'_id':'$restaurant_id','cuisine':'$cuisine','name':'$name', 
                               'borough':'$borough',
                               'average': {'$avg':"$grades.score"}}}) #각 레스토랑의 평균 구함
pipelines.append({'$group': {'_id':{'cuisine':'$cuisine', 'name':'$name'},
                             'max_score':{'$max':'$average'}}})
#cuisine과 name으로 묶고, 평균의 최대값을 구함
pipelines.append({"$sort":{'_id.cuisine':1, 'max_score':-1}})
#정렬

pipelines.append({'$group':{'_id':'$_id.cuisine',
                            'name':{'$first':'$_id.name'},
                            'score':{'$first':'$max_score'}}})
#cuisine별로 제일 위의 값 1개를 가져옴






result = col.aggregate(pipelines)

for doc in result:
    pprint (doc)
    

{u'_id': u'Chilean', u'name': u'Corwn Fried Chicken And Fish', u'score': 6.0}
{u'_id': u'Czech',
 u'name': u'Bohemian Beer Garden',
 u'score': 12.166666666666666}
{u'_id': u'Nuts/Confectionary', u'name': u'Laduree', u'score': 12.0}
{u'_id': u'Caf\xc3\xa9/Coffee/Tea',
 u'name': u"Mike'S Coffee Shop",
 u'score': 13.25}
{u'_id': u'Salads', u'name': u'Just Salad', u'score': 13.8}
{u'_id': u'Hotdogs', u'name': u"Nathan'S Famous", u'score': 14.0}
{u'_id': u'Egyptian', u'name': u'Zoe Cafe', u'score': 14.5}
{u'_id': u'Portuguese', u'name': u'Pao', u'score': 16.0}
{u'_id': u'Southwestern', u'name': u'The Cardinal', u'score': 16.8}
{u'_id': u'Ethiopian',
 u'name': u'Lalibela Ethiopian Restaurant',
 u'score': 17.5}
{u'_id': u'Afghan', u'name': u'Bakhter Afghan Halal Kabab', u'score': 18.25}
{u'_id': u'Hawaiian', u'name': u'Makana', u'score': 20.0}
{u'_id': u'Hotdogs/Pretzels', u'name': u"Auntie Anne'S", u'score': 21.5}
{u'_id': u'Bottled beverages, including water, sodas, juices, etc.',
 u'name':